<H1>Memory가 제대로 적용되지 않은 듯 함. 실패한 버전</H1>
<H2>이유: Memory에는 Input이 1개만 들어갈 수 있음</H2>

In [1]:
from dotenv import load_dotenv
load_dotenv('../config.env')

True

In [2]:
import sys
import pandas as pd

pd.set_option('display.width', sys.maxsize)

In [3]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from handler.custom_streaming_handler import CustomStreamingCallbackHandler
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0.5,
                  streaming=True,
                  callbacks=[CustomStreamingCallbackHandler()],
                  verbose=True)

memory = ConversationBufferMemory(memory_key="chat_history",
                                  return_messages=True)

In [4]:
# 1. template를 만들고, system message prompt로 변환한다.
info_template = """
너는 회사의 신입 사원을 뽑는 면접관이다.
주어지는 회사 이름, 직무명, 모집 공고, 그리고 면접자의 자기소개서를 바탕으로 면접자의 답변에 대해서 평가해야 한다.

먼저 회사와 직무에 대한 정보는 다음과 같다.
```
회사명: {company}
직무명: {job}
모집 공고: {job_requirements}
```

다음은 면접자의 자기소개서이다.
```
{cover_letter}
```

평가는 좋은 점과 아쉬운 점으로 나누어서 평가하라.
답변은 아래의 형식을 따라서 출력하라.
```
좋은 점:
 - 좋은 점들

아쉬운 점:
 - 아쉬운 점들
```
"""

info_prompt = PromptTemplate(
    template=info_template,
    input_variables=["company", "job", "job_requirements", "cover_letter"]
)

initial_system_message_prompt = SystemMessagePromptTemplate(
    prompt=info_prompt
)

In [5]:
# 2. human_template를 만들고, human message prompt로 변환한다.
user_input_template ="""
{chat_history}
{user_input}
"""

user_input_prompt = PromptTemplate(
    template=user_input_template,
    input_variables=["chat_history", "user_input"],
)

initial_human_message_prompt = HumanMessagePromptTemplate(
    prompt=user_input_prompt
)

In [6]:
# 3. system message prompt와 human message prompt를 이용해서 chat prompt를 만든다.
initial_chat_prompt = ChatPromptTemplate.from_messages([initial_system_message_prompt, initial_human_message_prompt])

In [7]:
# 4. Chain으로 llm과 prompt 엮기
info_chain = LLMChain(llm=chat,
                      prompt=initial_chat_prompt,
                      verbose=True,
                      memory=memory
)

In [8]:
# 5. 입력 및 실행
input_company = "LG CNS"
input_job = "DX Engineer"

input_job_requirements = """
전사: DX Engineer
전공: 전공 무관 (*) 단, 컴퓨터공학/정보통신공학/산업공학/전자전기공학/기계공학 등 이공계열은 우대합니다
근무지: 서울특별시
인원: 00명
상세내용:
- 이런 일을 해요
  - 다양한 산업 분야에서 DX 신기술로 고객 맞춤형 서비스를 제공하며, 고객 비즈니스에 가치를 더하는 일을 합니다
  - 기존에 없던 새로운 시스템을 만들어내고, 시스템을 안정적으로 운영해냅니다.
  - 디지털금융, 지능형 정부, 스마트시티, 스마트 교통 서비스 등 편리하고 보다 나은 삶을 위한 변화를 이끌어갑니다.
  - 국내외 고객에게 DX환경에 적합한 클라우드 인프라 설계/구축/컨설팅/운영 서비스를 제공합니다.
  - 국내 및 해외(미국, 유럽, 중국 등) 데이터센터와 클라우드 인프라 기술을 이용하여, 전 세계를 하나로 연결하는 서비스를 제공합니다.

- 이런 분이면 더 좋겠어요!
  - 새로운 시도를 즐기고, IT를 통해 세상의 변화를 이끌어가고 싶은 분
  - 끊임없이 신기술을 익히고 적용하며 성장하고 싶으신 분
  - 비즈니스를 이해하고 더 편리한 서비스를 제공하는데 관심이 많으신 분
"""

input_cover_letter = """
1. LG CNS에서 근무하고 싶은 이유에 대해서 관련된 경험 및 역량 등 자신을 어필할 수 있는 내용을 기반으로 자유롭게 기술하시기 바랍니다.
답변:
저는 기술적 도전을 통해 기대를 뛰어넘는 결과를 만들어서 사용자에게 더욱 편리한 서비스를 제공하는 개발자가 되고 싶습니다. LG CNS는 최초로 자바를 개발언어로 사용하는 도전을 통해 고객의 니즈를 바탕으로 더욱 뛰어난 결과를 만들어낸 경험이 있고,카카오뱅크 전산시스템을 통해 인터넷 은행의 시대를 열었습니다. 또한 현재 AI,클라우드,IoT,블록체인 등 4차산업기술을 선도하고 있고, 그 중에서도 특히 클라우드 전환과 MSP 사업에 앞서고 가장 많은 AM프로젝트를 수행하는 등 클라우드 기술을 기반으로 디지털 혁신에 앞장서고 있습니다.

저는 프로젝트를 진행하며 요구사항을 직접 세우고 분석하는 과정을 통해 애플리케이션 설계,데이터 모델링,백엔드 등에 대한 지식을 갖출 수 있었고, 앱,웹,AI모델링과 같은 다양한 기술에 도전하면서 새로운 기술에 대한 적응력을 얻었습니다. 이러한 경험과 LG CNS의 혁신 정신을 바탕으로 신기술을 배우고 활용하여 고객에게 최고의 서비스를 제공하고자 합니다.


2. 지원분야와 관련된 지식이나 경험을 기재하여 주십시오.
답변:
"식사 추천 앱"
사용자의 식습관을 토대로 음식을 추천해주는 안드로이드 앱 프로젝트로, 설문 조사를 통해 약 200여 건의 식사 기록을 확보하고 AWS의 SaaS형태의 학습 서비스를 이용하여 학습 한 뒤, AWS 클라우드 서비스를 기반으로 서버를 구축한 서비스입니다. 저는 AWS 서비스와 API를 통해 통신하며 앱에서 음식 추천 목록과 선택된 음식의 정보를 띄워주고, 게시판 및 댓글 기능을 지원하는 등 앱의 기능을 구현하는 역할을 담당했습니다.

이 프로젝트를 진행하면서 사용자의 시점에서 직접 요구 사항을 작성하고, 개발자의 시점에서 요구사항을 만족하기 위해 필요한 기술과 리스크를 예측해볼 수 있었습니다. 또한 API를 통한 서버와의 통신 시 주고 받아야 할 요청 사항과 응답 내용에 대한 협의 과정을 겪을 수 있었고, 짧은 시간 내에 프로젝트에 적용하기 위해서 안드로이드 docs를 찾아가며 지식을 습득하고 적용했던 과정을 통해서 새로운 지식 습득에 대한 자신감을 가질 수 있었습니다.


3. 10년 뒤 본인이 기대하는 모습을 기재하여 주십시오.
답변:
지금의 DX는 단순히 종이 신문을 스마트폰으로 보여주고, 쇼핑몰을 인터넷으로 옮기는 것이 아니라 4차 산업 기술을 인프라나 시스템에 적용하고 서비스와 결합하고 있습니다. 하지만 4차 산업 기술들은 이제야 제대로 된 발전 과정에 들어섰을 뿐, 진짜 변화는 지금부터라고 생각합니다. 그 중에서도 클라우드 분야는 앞으로는 개인 컴퓨터 없이 모니터와 마우스만 가지고 컴퓨팅 자원을 빌려서 사용하는 시대가 올 정도로 발전하게 될 것이라고 생각합니다.

이런 미래를 앞두고 있는 상황에서 저는 클라우드 기반의 인프라 지식 또한 DX 엔지니어로써 갖추어야 할 소양이라고 생각합니다. 저는 다양한 프로젝트에 참여하여 경험을 쌓으면서 다양한 분야의 업무와 시스템 구조를 파악하며 각 산업에 대한 지식을 갖추고, 동시에 지속적인 자기 계발을 통해 개발 기술과 함께 클라우드 지식까지 갖춘 전문가가 되어 고객의 기대를 뛰어넘는 DX 경험을 제공하는 최고의 인재가 되겠습니다.
"""

info_chain.run(company=input_company,
               job=input_job,
               job_requirements=input_job_requirements,
               cover_letter=input_cover_letter,
               user_input="""
               면접 질문: 팀원과의 갈등이 있을 때 해결한 경험이 있는가?
               면접 답변: 안드로이드 앱 프로젝트를 계획할 당시에 추천된 음식을 실제로 먹었는가를 확인하는 방식에 대해서 의견 차이가 있었던 경험이 있습니다.
                       저는 처음에 먹을래요 버튼을 추가하자는 의견을, 다른 팀원은 단순히 해당 음식을 클릭했을 때 실제로 먹었다고 판단하자는 의견을 냈었는데
                       서로의 의견에 대한 장단점을 이야기하다 보니 아예 누르지 않고 앱을 끌 수도 있다는 새로운 문제점을 발견하게 되었고,
                       그 문제점까지 포함해서 해결할 방법을 도출하게 되었습니다.
                       이런 경험을 통해서 의견을 주고받으면서 고민하는 과정의 중요성에 알게 되었습니다.
               """
)



> Entering new  chain...
Prompt after formatting:
System: 
너는 회사의 신입 사원을 뽑는 면접관이다.
주어지는 회사 이름, 직무명, 모집 공고, 그리고 면접자의 자기소개서를 바탕으로 면접자의 답변에 대해서 평가해야 한다.

먼저 회사와 직무에 대한 정보는 다음과 같다.
```
회사명: LG CNS
직무명: DX Engineer
모집 공고: 
전사: DX Engineer
전공: 전공 무관 (*) 단, 컴퓨터공학/정보통신공학/산업공학/전자전기공학/기계공학 등 이공계열은 우대합니다
근무지: 서울특별시
인원: 00명
상세내용:
- 이런 일을 해요
  - 다양한 산업 분야에서 DX 신기술로 고객 맞춤형 서비스를 제공하며, 고객 비즈니스에 가치를 더하는 일을 합니다
  - 기존에 없던 새로운 시스템을 만들어내고, 시스템을 안정적으로 운영해냅니다.
  - 디지털금융, 지능형 정부, 스마트시티, 스마트 교통 서비스 등 편리하고 보다 나은 삶을 위한 변화를 이끌어갑니다.
  - 국내외 고객에게 DX환경에 적합한 클라우드 인프라 설계/구축/컨설팅/운영 서비스를 제공합니다.
  - 국내 및 해외(미국, 유럽, 중국 등) 데이터센터와 클라우드 인프라 기술을 이용하여, 전 세계를 하나로 연결하는 서비스를 제공합니다.

- 이런 분이면 더 좋겠어요!
  - 새로운 시도를 즐기고, IT를 통해 세상의 변화를 이끌어가고 싶은 분
  - 끊임없이 신기술을 익히고 적용하며 성장하고 싶으신 분
  - 비즈니스를 이해하고 더 편리한 서비스를 제공하는데 관심이 많으신 분

```

다음은 면접자의 자기소개서이다.
```

1. LG CNS에서 근무하고 싶은 이유에 대해서 관련된 경험 및 역량 등 자신을 어필할 수 있는 내용을 기반으로 자유롭게 기술하시기 바랍니다.
답변:
저는 기술적 도전을 통해 기대를 뛰어넘는 결과를 만들어서 사용자에게 더욱 편리한 서비스를 제공하는 개발자가 되고 싶습니다. LG CNS는 최초로 자바를 개발언어로 사용하는 도전을 

ValueError: One input key expected got ['job_requirements', 'user_input', 'job', 'cover_letter', 'company']

In [10]:
# # 1. template를 만들고, system message prompt로 변환한다.
# follow_up_template = """
# {chat_history}
# """
#
# follow_up_prompt = PromptTemplate(
#     template=follow_up_template,
#     input_variables=["chat_history"]
# )
#
# follow_up_system_message_prompt = SystemMessagePromptTemplate(
#     prompt=follow_up_prompt
# )

In [12]:
# # 2. human_template를 만들고, human message prompt로 변환한다.
# user_input_template ="""
# {user_input}
# """
#
# user_input_prompt = PromptTemplate(
#     template=user_input_template,
#     input_variables=["user_input"]
# )
#
# follow_up_human_message_prompt = HumanMessagePromptTemplate(
#     prompt=user_input_prompt
# )

In [13]:
# # 3. system message prompt와 human message prompt를 이용해서 chat prompt를 만든다.
# follow_up_chat_prompt = ChatPromptTemplate.from_messages([follow_up_system_message_prompt, follow_up_human_message_prompt])

In [14]:
# # 4. Chain으로 llm과 prompt 엮기
# follow_up_chain = LLMChain(llm=chat,
#                            prompt=follow_up_chat_prompt,
#                            verbose=True,
#                            memory=memory
# )

In [16]:
# # 5. 입력 및 실행
# follow_up_chain.run(user_input="분석한 내용을 바탕으로 follow up question을 해보아라.")



> Entering new  chain...
Prompt after formatting:
System: 
[]

Human: 
분석한 내용을 바탕으로 follow up question을 해보아라.

Run when LLM starts running.
1. 분석 결과에 따르면 어떤 요인이 가장 큰 영향을 미치는 것으로 나타났나요?
2. 분석 결과를 토대로 어떤 전략이나 조치를 취할 수 있을까요?
3. 분석 결과를 통해 어떤 인사이트나 통찰력을 얻을 수 있었나요?
4. 분석 결과가 예상과 일치했는지 확인할 수 있는 방법이 있을까요?
5. 분석 결과를 기반으로 어떤 가설을 세울 수 있을까요?

> Finished chain.


'1. 분석 결과에 따르면 어떤 요인이 가장 큰 영향을 미치는 것으로 나타났나요?\n2. 분석 결과를 토대로 어떤 전략이나 조치를 취할 수 있을까요?\n3. 분석 결과를 통해 어떤 인사이트나 통찰력을 얻을 수 있었나요?\n4. 분석 결과가 예상과 일치했는지 확인할 수 있는 방법이 있을까요?\n5. 분석 결과를 기반으로 어떤 가설을 세울 수 있을까요?'

In [17]:
info_template = """
너는 회사의 신입 사원을 뽑는 면접관이다.
주어지는 회사 이름, 직무명, 모집 공고, 그리고 면접자의 자기소개서를 바탕으로 면접자의 답변을 평가한 내용을 이용하여 follow up question 1개를 만들어야 한다.

먼저 회사와 직무에 대한 정보는 다음과 같다.
```
회사명: {company}
직무명: {job}
모집 공고: {job_requirements}
```

다음은 면접자의 자기소개서이다.
```
{cover_letter}
```

새로운 질문은 아래의 형식을 따라서 출력하라.
```
심화 질문:

```
"""

info_prompt = PromptTemplate(
    template=info_template,
    input_variables=["company", "job", "job_requirements", "cover_letter"]
)

initial_system_message_prompt = SystemMessagePromptTemplate(
    prompt=info_prompt
)

In [18]:
# 2. human_template를 만들고, human message prompt로 변환한다.
user_input_template ="""
{chat_history}
{user_input}
"""

user_input_prompt = PromptTemplate(
    template=user_input_template,
    input_variables=["chat_history", "user_input"],
)

initial_human_message_prompt = HumanMessagePromptTemplate(
    prompt=user_input_prompt
)

In [19]:
# 3. system message prompt와 human message prompt를 이용해서 chat prompt를 만든다.
initial_chat_prompt = ChatPromptTemplate.from_messages([initial_system_message_prompt, initial_human_message_prompt])

In [20]:
# 4. Chain으로 llm과 prompt 엮기
info_chain = LLMChain(llm=chat,
                      prompt=initial_chat_prompt,
                      verbose=True,
                      memory=memory
)

In [21]:
# 5. 입력 및 실행
input_company = "LG CNS"
input_job = "DX Engineer"

input_job_requirements = """
전사: DX Engineer
전공: 전공 무관 (*) 단, 컴퓨터공학/정보통신공학/산업공학/전자전기공학/기계공학 등 이공계열은 우대합니다
근무지: 서울특별시
인원: 00명
상세내용:
- 이런 일을 해요
  - 다양한 산업 분야에서 DX 신기술로 고객 맞춤형 서비스를 제공하며, 고객 비즈니스에 가치를 더하는 일을 합니다
  - 기존에 없던 새로운 시스템을 만들어내고, 시스템을 안정적으로 운영해냅니다.
  - 디지털금융, 지능형 정부, 스마트시티, 스마트 교통 서비스 등 편리하고 보다 나은 삶을 위한 변화를 이끌어갑니다.
  - 국내외 고객에게 DX환경에 적합한 클라우드 인프라 설계/구축/컨설팅/운영 서비스를 제공합니다.
  - 국내 및 해외(미국, 유럽, 중국 등) 데이터센터와 클라우드 인프라 기술을 이용하여, 전 세계를 하나로 연결하는 서비스를 제공합니다.

- 이런 분이면 더 좋겠어요!
  - 새로운 시도를 즐기고, IT를 통해 세상의 변화를 이끌어가고 싶은 분
  - 끊임없이 신기술을 익히고 적용하며 성장하고 싶으신 분
  - 비즈니스를 이해하고 더 편리한 서비스를 제공하는데 관심이 많으신 분
"""

input_cover_letter = """
1. LG CNS에서 근무하고 싶은 이유에 대해서 관련된 경험 및 역량 등 자신을 어필할 수 있는 내용을 기반으로 자유롭게 기술하시기 바랍니다.
답변:
저는 기술적 도전을 통해 기대를 뛰어넘는 결과를 만들어서 사용자에게 더욱 편리한 서비스를 제공하는 개발자가 되고 싶습니다. LG CNS는 최초로 자바를 개발언어로 사용하는 도전을 통해 고객의 니즈를 바탕으로 더욱 뛰어난 결과를 만들어낸 경험이 있고,카카오뱅크 전산시스템을 통해 인터넷 은행의 시대를 열었습니다. 또한 현재 AI,클라우드,IoT,블록체인 등 4차산업기술을 선도하고 있고, 그 중에서도 특히 클라우드 전환과 MSP 사업에 앞서고 가장 많은 AM프로젝트를 수행하는 등 클라우드 기술을 기반으로 디지털 혁신에 앞장서고 있습니다.

저는 프로젝트를 진행하며 요구사항을 직접 세우고 분석하는 과정을 통해 애플리케이션 설계,데이터 모델링,백엔드 등에 대한 지식을 갖출 수 있었고, 앱,웹,AI모델링과 같은 다양한 기술에 도전하면서 새로운 기술에 대한 적응력을 얻었습니다. 이러한 경험과 LG CNS의 혁신 정신을 바탕으로 신기술을 배우고 활용하여 고객에게 최고의 서비스를 제공하고자 합니다.


2. 지원분야와 관련된 지식이나 경험을 기재하여 주십시오.
답변:
"식사 추천 앱"
사용자의 식습관을 토대로 음식을 추천해주는 안드로이드 앱 프로젝트로, 설문 조사를 통해 약 200여 건의 식사 기록을 확보하고 AWS의 SaaS형태의 학습 서비스를 이용하여 학습 한 뒤, AWS 클라우드 서비스를 기반으로 서버를 구축한 서비스입니다. 저는 AWS 서비스와 API를 통해 통신하며 앱에서 음식 추천 목록과 선택된 음식의 정보를 띄워주고, 게시판 및 댓글 기능을 지원하는 등 앱의 기능을 구현하는 역할을 담당했습니다.

이 프로젝트를 진행하면서 사용자의 시점에서 직접 요구 사항을 작성하고, 개발자의 시점에서 요구사항을 만족하기 위해 필요한 기술과 리스크를 예측해볼 수 있었습니다. 또한 API를 통한 서버와의 통신 시 주고 받아야 할 요청 사항과 응답 내용에 대한 협의 과정을 겪을 수 있었고, 짧은 시간 내에 프로젝트에 적용하기 위해서 안드로이드 docs를 찾아가며 지식을 습득하고 적용했던 과정을 통해서 새로운 지식 습득에 대한 자신감을 가질 수 있었습니다.


3. 10년 뒤 본인이 기대하는 모습을 기재하여 주십시오.
답변:
지금의 DX는 단순히 종이 신문을 스마트폰으로 보여주고, 쇼핑몰을 인터넷으로 옮기는 것이 아니라 4차 산업 기술을 인프라나 시스템에 적용하고 서비스와 결합하고 있습니다. 하지만 4차 산업 기술들은 이제야 제대로 된 발전 과정에 들어섰을 뿐, 진짜 변화는 지금부터라고 생각합니다. 그 중에서도 클라우드 분야는 앞으로는 개인 컴퓨터 없이 모니터와 마우스만 가지고 컴퓨팅 자원을 빌려서 사용하는 시대가 올 정도로 발전하게 될 것이라고 생각합니다.

이런 미래를 앞두고 있는 상황에서 저는 클라우드 기반의 인프라 지식 또한 DX 엔지니어로써 갖추어야 할 소양이라고 생각합니다. 저는 다양한 프로젝트에 참여하여 경험을 쌓으면서 다양한 분야의 업무와 시스템 구조를 파악하며 각 산업에 대한 지식을 갖추고, 동시에 지속적인 자기 계발을 통해 개발 기술과 함께 클라우드 지식까지 갖춘 전문가가 되어 고객의 기대를 뛰어넘는 DX 경험을 제공하는 최고의 인재가 되겠습니다.
"""

info_chain.run(company=input_company,
               job=input_job,
               job_requirements=input_job_requirements,
               cover_letter=input_cover_letter,
               user_input="""
               면접관의 분석 중 아쉬운 점을 이용하여 follow up question을 1개 제시하라.
               """
)



> Entering new  chain...
Prompt after formatting:
System: 
너는 회사의 신입 사원을 뽑는 면접관이다.
주어지는 회사 이름, 직무명, 모집 공고, 그리고 면접자의 자기소개서를 바탕으로 면접자의 답변을 평가한 내용을 이용하여 follow up question 1개를 만들어야 한다.

먼저 회사와 직무에 대한 정보는 다음과 같다.
```
회사명: LG CNS
직무명: DX Engineer
모집 공고: 
전사: DX Engineer
전공: 전공 무관 (*) 단, 컴퓨터공학/정보통신공학/산업공학/전자전기공학/기계공학 등 이공계열은 우대합니다
근무지: 서울특별시
인원: 00명
상세내용:
- 이런 일을 해요
  - 다양한 산업 분야에서 DX 신기술로 고객 맞춤형 서비스를 제공하며, 고객 비즈니스에 가치를 더하는 일을 합니다
  - 기존에 없던 새로운 시스템을 만들어내고, 시스템을 안정적으로 운영해냅니다.
  - 디지털금융, 지능형 정부, 스마트시티, 스마트 교통 서비스 등 편리하고 보다 나은 삶을 위한 변화를 이끌어갑니다.
  - 국내외 고객에게 DX환경에 적합한 클라우드 인프라 설계/구축/컨설팅/운영 서비스를 제공합니다.
  - 국내 및 해외(미국, 유럽, 중국 등) 데이터센터와 클라우드 인프라 기술을 이용하여, 전 세계를 하나로 연결하는 서비스를 제공합니다.

- 이런 분이면 더 좋겠어요!
  - 새로운 시도를 즐기고, IT를 통해 세상의 변화를 이끌어가고 싶은 분
  - 끊임없이 신기술을 익히고 적용하며 성장하고 싶으신 분
  - 비즈니스를 이해하고 더 편리한 서비스를 제공하는데 관심이 많으신 분

```

다음은 면접자의 자기소개서이다.
```

1. LG CNS에서 근무하고 싶은 이유에 대해서 관련된 경험 및 역량 등 자신을 어필할 수 있는 내용을 기반으로 자유롭게 기술하시기 바랍니다.
답변:
저는 기술적 도전을 통해 기대를 뛰어넘는 결과를 만들어서 사용자에게 더욱 편리한 서비스를 제공하는 개발자가 되고 싶습니다.

ValueError: One input key expected got ['job_requirements', 'user_input', 'job', 'cover_letter', 'company']